In [53]:

from sklearn.ensemble import RandomForestClassifier
from agoda_cancellation_estimator import AgodaCancellationEstimator
from agoda_cancellation_prediction import load_data
import pandas as pd
import numpy as np

In [13]:
full_data = pd.read_csv(r"C:\Users\97250\PycharmProjects\IML.HUJI\datasets\agoda_cancellation_train.csv").dropna().drop_duplicates()
full_data.head()

,h_booking_id,booking_datetime,checkin_date,checkout_date,hotel_id,hotel_country_code,hotel_live_date,hotel_star_rating,accommadation_type_name,charge_option,...,request_highfloor,request_largebed,request_twinbeds,request_airport,request_earlycheckin,cancellation_datetime,hotel_area_code,hotel_brand_code,hotel_chain_code,hotel_city_code
16,-9216487696802692260,2018-08-28 22:40:00,2018-09-29 00:00:00,2018-09-30 00:00:00,2559209,JP,2017-07-21 18:32:00,3.0,Hotel,Pay Now,...,0.0,0.0,1.0,0.0,0.0,2018-08-28,5092,366.0,618.0,1403
19,-9215494322833174318,2018-05-27 17:15:00,2018-08-20 00:00:00,2018-08-21 00:00:00,11934,NZ,2009-06-28 02:02:00,4.0,Hotel,Pay Later,...,0.0,0.0,1.0,0.0,0.0,2018-07-13,3450,159.0,232.0,2158
22,-9215072291520077707,2018-05-31 22:43:00,2018-09-21 00:00:00,2018-09-22 00:00:00,108170,TH,2009-06-28 02:02:00,3.0,Hotel,Pay Later,...,1.0,1.0,0.0,0.0,0.0,2018-06-19,662,442.0,600.0,447
98,-9191194571019688471,2018-06-18 09:47:00,2018-09-07 00:00:00,2018-09-09 00:00:00,2411232,US,2017-08-30 00:49:00,4.0,Hotel,Pay Later,...,0.0,0.0,0.0,0.0,0.0,2018-09-05,5540,753.0,548.0,202
137,-9179243281126879585,2018-03-13 16:09:00,2018-08-24 00:00:00,2018-08-28 00:00:00,1899389,JP,2017-01-27 09:03:00,4.0,Hotel,Pay Later,...,1.0,0.0,1.0,0.0,0.0,2018-03-23,643,95.0,254.0,2249


In [40]:

full_data.accommadation_type_name = full_data.accommadation_type_name.astype('category')
full_data.customer_nationality = full_data.customer_nationality.astype(str).astype('category')
full_data.hotel_id = full_data.hotel_id.astype('category')
full_data['customer_nationality_codes'] = full_data.customer_nationality.cat.codes
full_data['accommadation_type_name_codes'] = full_data.accommadation_type_name.cat.codes
full_data['hotel_id_codes'] = full_data.hotel_id.cat.codes

In [63]:
full_data['label'] = full_data.cancellation_datetime.map(lambda x: x > '2018-08-07' and x < '2018-08-20')
full_data.loc[:, ['cancellation_datetime', 'label']]

,cancellation_datetime,label
16,2018-08-28,False
19,2018-07-13,False
22,2018-06-19,False
98,2018-09-05,False
137,2018-03-23,False
...,...,...
58548,2018-03-09,False
58556,2018-07-22,False
58572,2018-07-18,False
58602,2018-05-20,False


In [86]:
train_ratio = 0.75
rand_nums = np.random.random_sample(len(full_data))
full_data['train'] =  rand_nums <= train_ratio
full_data.train.value_counts()
train_df = full_data[full_data.train]
test_df = full_data[~full_data.train]
len(train_df), len(test_df), len(full_data)

(1794, 571, 2365)

In [74]:
features_cols = [#"h_booking_id",
                      "hotel_id",
                      "accommadation_type_name_codes",
                      "hotel_star_rating",
                      "customer_nationality_codes"]
train_features = train_df[features_cols]
test_features = test_df[features_cols]
train_features.head()


,hotel_id,accommadation_type_name_codes,hotel_star_rating,customer_nationality_codes
16,2559209,5,3.0,54
19,11934,5,4.0,36
22,108170,5,3.0,55
98,2411232,5,4.0,61
137,1899389,5,4.0,20


In [42]:
# features = full_data[[#"h_booking_id",
#                       "hotel_id",
#                       "accommadation_type_name_codes",
#                       "hotel_star_rating",
#                       "customer_nationality_codes"]]
# features.head()


,hotel_id,accommadation_type_name_codes,hotel_star_rating,customer_nationality_codes
16,2559209,5,3.0,54
19,11934,5,4.0,36
22,108170,5,3.0,55
98,2411232,5,4.0,61
137,1899389,5,4.0,20


In [75]:
train_labels = train_df["cancellation_datetime"]
test_labels = test_df["cancellation_datetime"]
train_labels.head()

16     2018-08-28
19     2018-07-13
22     2018-06-19
98     2018-09-05
137    2018-03-23
Name: cancellation_datetime, dtype: object

In [11]:
full_data['cancellation_datetime']

16       2018-08-28
19       2018-07-13
22       2018-06-19
98       2018-09-05
137      2018-03-23
            ...    
58548    2018-03-09
58556    2018-07-22
58572    2018-07-18
58602    2018-05-20
58626    2018-08-23
Name: cancellation_datetime, Length: 2365, dtype: object

In [79]:
random_classifier = RandomForestClassifier()

In [84]:
y_train = train_df.label.values
X_train = train_features.values
y_test = test_df.label.values
X_test = test_features.values

In [81]:
model = random_classifier.fit(X_train,y_train)

In [82]:
np.bincount(model.predict(X_train))

array([1549,  177], dtype=int64)

In [ ]:
model.predict(X_test)

In [85]:
np.bincount(model.predict(X_test))

array([611,  28], dtype=int64)

In [67]:
full_data.label.value_counts()

False    2110
True      255
Name: label, dtype: int64

In [61]:
full_data[(full_data.cancellation_datetime > '2018-08-07') & (full_data.cancellation_datetime < '2018-08-20')]

,h_booking_id,booking_datetime,checkin_date,checkout_date,hotel_id,hotel_country_code,hotel_live_date,hotel_star_rating,accommadation_type_name,charge_option,...,request_earlycheckin,cancellation_datetime,hotel_area_code,hotel_brand_code,hotel_chain_code,hotel_city_code,label,customer_nationality_codes,accommadation_type_name_codes,hotel_id_codes
138,-9178271518124899741,2018-06-13 01:39:00,2018-09-20 00:00:00,2018-09-24 00:00:00,2863,GB,2011-08-31 16:49:00,3.0,Hostel,Pay Now,...,0.0,2018-08-13,5134,131.0,271.0,2538,False,58,4,23
200,-9158997573325902499,2018-08-19 06:24:00,2018-09-15 00:00:00,2018-09-17 00:00:00,3129374,AU,2017-11-06 16:37:00,4.5,Hotel,Pay Now,...,0.0,2018-08-19,4682,63.0,291.0,1584,False,1,5,1347
225,-9150457930144649121,2018-07-24 01:47:00,2018-08-18 00:00:00,2018-08-19 00:00:00,1602888,IN,2017-03-17 11:34:00,5.0,Hotel,Pay Later,...,0.0,2018-08-08,5679,282.0,55.0,2796,False,21,5,876
235,-9148315339177572723,2018-08-13 22:49:00,2018-08-18 00:00:00,2018-08-19 00:00:00,4910396,MY,2018-04-13 19:52:00,2.0,Hotel,Pay Later,...,0.0,2018-08-16,5596,745.0,94.0,2797,False,32,5,1597
412,-9093276767136234643,2018-06-27 23:39:00,2018-09-19 00:00:00,2018-09-20 00:00:00,168985,JP,2009-12-19 08:20:00,3.5,Hotel,Pay Later,...,0.0,2018-08-19,260,918.0,587.0,1014,False,58,5,349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56263,8466705461484802320,2018-07-27 16:15:00,2018-08-24 00:00:00,2018-08-26 00:00:00,1105052,JP,2015-11-25 11:59:00,3.0,Hotel,Pay Later,...,0.0,2018-08-16,4975,922.0,587.0,183,False,58,5,770
56332,8488847189900316785,2018-07-10 12:45:00,2018-08-25 00:00:00,2018-08-26 00:00:00,9358,JP,2009-06-28 02:02:00,4.0,Hotel,Pay Later,...,0.0,2018-08-13,117,463.0,103.0,1403,False,32,5,69
56435,8520771494965438804,2018-08-18 21:31:00,2018-08-20 00:00:00,2018-08-21 00:00:00,240484,JP,2010-11-18 07:08:00,3.5,Hotel,Pay Now,...,0.0,2018-08-19,1986,777.0,181.0,1783,False,20,5,427
56514,8546579788317624088,2018-08-02 11:58:00,2018-09-13 00:00:00,2018-09-18 00:00:00,3320368,TH,2017-11-10 09:25:00,4.0,Hotel,Pay Later,...,0.0,2018-08-16,3747,278.0,473.0,2477,False,54,5,1450
